# Mobotix

In [1]:
import sage_data_client
import urllib
import os
import glob

import cv2

In [2]:
data_dir = '/Users/bhupendra/data/mobotest/'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [7]:
df = sage_data_client.query(
    start="20230101-20:25:00",
    end="20230522-20:25:10",
    filter={
        "plugin": "*mobotix-scan.*",
         "name": "upload"
        #"vsn" : "V008"
    }
)

df

,timestamp,name,value,meta.filename,meta.host,meta.loop_num,meta.node,meta.plugin,meta.position,meta.task,meta.vsn,meta.zone,meta.job
0,2023-05-04 18:51:09.574221815+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,3840x1080_position0.jpg,0000e45f01453930.ws-rpi,1,000048b02d3ae2f4,registry.sagecontinuum.org/bhupendraraut/mobot...,0,mobo-nonpt-sample,W084,shield,NaN
1,2023-05-04 19:36:30.308646501+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,3840x1080_position0.jpg,0000e45f01453930.ws-rpi,10,000048b02d3ae2f4,registry.sagecontinuum.org/bhupendraraut/mobot...,0,mobo-nonpt-sample,W084,shield,NaN
2,2023-05-04 19:41:32.628397686+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,3840x1080_position0.jpg,0000e45f01453930.ws-rpi,11,000048b02d3ae2f4,registry.sagecontinuum.org/bhupendraraut/mobot...,0,mobo-nonpt-sample,W084,shield,NaN
3,2023-05-04 19:46:34.808443352+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,3840x1080_position0.jpg,0000e45f01453930.ws-rpi,12,000048b02d3ae2f4,registry.sagecontinuum.org/bhupendraraut/mobot...,0,mobo-nonpt-sample,W084,shield,NaN
4,2023-05-04 19:51:37.144355997+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,3840x1080_position0.jpg,0000e45f01453930.ws-rpi,13,000048b02d3ae2f4,registry.sagecontinuum.org/bhupendraraut/mobot...,0,mobo-nonpt-sample,W084,shield,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4029050,2023-05-20 04:46:28.757161980+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,right_336x252_14bit.thermal_position9.raw,0000e45f01453930.ws-rpi,51,000048b02d3ae2f4,registry.sagecontinuum.org/bhupendraraut/mobot...,9,mobotix-scan,W084,shield,NaN
4029051,2023-05-19 22:50:51.972579566+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,right_336x252_14bit.thermal_position9.raw,0000e45f01453930.ws-rpi,6,000048b02d3ae2f4,registry.sagecontinuum.org/bhupendraraut/mobot...,9,mobotix-scan,W084,shield,NaN
4029052,2023-05-19 22:58:39.815606808+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,right_336x252_14bit.thermal_position9.raw,0000e45f01453930.ws-rpi,7,000048b02d3ae2f4,registry.sagecontinuum.org/bhupendraraut/mobot...,9,mobotix-scan,W084,shield,NaN
4029053,2023-05-19 23:06:31.521804390+00:00,upload,https://storage.sagecontinuum.org/api/v1/data/...,right_336x252_14bit.thermal_position9.raw,0000e45f01453930.ws-rpi,8,000048b02d3ae2f4,registry.sagecontinuum.org/bhupendraraut/mobot...,9,mobotix-scan,W084,shield,NaN


In [8]:
df.to_csv('/Users/bhupendra/data/mobo-meta.csv')

In [54]:
def download_files_beehive(df, file_ext):
    #get the required urls
    targets = []
    times = []
    for i in range(len(df)):
        if file_ext in df.iloc[i].value:
            targets.append(df.iloc[i].value)
            times.append(df.iloc[i].timestamp)
            
    targets
    # Download and rename
    for i in range(len(times)): 
        print(times[i])
        link = targets[i]
        fname = link.split('-')[-1]
        filename = os.path.join(data_dir,times[i].strftime('sage_mobotix_cam_%Y%m%d-%H%M%S_'+fname)) 
        urllib.request.urlretrieve(link, filename)

In [56]:
download_files_beehive(df, file_ext='jpg')
download_files_beehive(df, file_ext='csv')

2022-04-15 20:25:02.899345804+00:00
2022-04-15 20:25:07.123196393+00:00
2022-04-15 20:25:02.899345804+00:00
2022-04-15 20:25:07.123196393+00:00
2022-04-15 20:25:02.899345804+00:00
2022-04-15 20:25:07.123196393+00:00


# Show images

In [57]:
flist = glob.glob(data_dir+'*.jpg')
flist.sort()
flist


['/Users/bhupendra/data/mobotest/sage_mobotix_cam_20220415-150003_6144x2048.jpg',
 '/Users/bhupendra/data/mobotest/sage_mobotix_cam_20220415-200000_6144x2048.jpg',
 '/Users/bhupendra/data/mobotest/sage_mobotix_cam_20220415-201001_6144x2048.jpg',
 '/Users/bhupendra/data/mobotest/sage_mobotix_cam_20220415-202000_6144x2048.jpg',
 '/Users/bhupendra/data/mobotest/sage_mobotix_cam_20220415-202502_6144x2048.jpg',
 '/Users/bhupendra/data/mobotest/sage_mobotix_cam_20220415-202507_6144x2048.jpg',
 '/Users/bhupendra/data/mobotest/sage_mobotix_cam_20220415-203004_6144x2048.jpg',
 '/Users/bhupendra/data/mobotest/sage_mobotix_cam_20220415-220001_6144x2048.jpg']

In [60]:
img = cv2.imread(flist[0])
cv2.imshow('image' ,img)